# Git - Control de Versiones
*Sofía García Salas*

*2021-03-02*

***

Estas son mis notas sobre la clase de MIT [The Missing Semester](https://missing.csail.mit.edu/2020/version-control/).

## Control de versiones
`Git` es un sistema de control de versiones que permite documentar los cambios a código u otro documentos y folders. También facilitan el trabajo en equipo.

Estos sistemas almacenan la historia de cambios tomando una fotografía al estado de los archivos y folders. No solo tienen la historia de los cambios a estos archivos sino que también almacenan metadata (e.g. quién realizó un cambio, en qué momento).

### Utilidad
Sirve no solo en proyectos grandes, sino en proyecto personales para poder revisar versiones antiguos del código, ver por qué se cambio algo (viendo commit messages), trabajar en paralelo con distintas branches para el desarrollo, etc. 

### Git
Se ha vuelto el sistema de control de versiones más popular.

Se interfaz no es muy amigable con el usuario, por lo que el profesor recomienda empezar aprendiendo su diseño e ideas. Al entender esto, será más fácil utilizar la interfaz y resolver problemas.

## Modelo de datos de Git
### Modelo de folders y archivos
Es una colección de archivos y folders dentro de un directorio maestro.
Un folder es denominado como `tree` y un archivo como `blob`.

```
root
|-foo (tree)
| |-bar.txt (blob)
|-baz.txt

```

El folder `root`es el directorio al que se le está dando seguimiento.

### Modelo de historia
Una forma de crear un modelo de historia es tomar una fotografía de los folders y archivos en el directorio. La historia se vuelve una secuencia lineal de estas fotografías. Es muy similar a tener copias de folders con marcas de tiempo.

Git utilizado un modelo más complejo llamado **Directed acyclic graph**. 

Con este modelo uno puede separar versiones (`fork`) para trabajar en ramas (`branches`) en paralelo sin interferencia; por ejemplo, trabajar es una nueva característica y en paralelo arreglar un desperfecto en el código en producción. 
Estas ramas después se pueden unir (`merge`) y la nueva fotografía tendrá como referencia las dos versiones que se unieron.



![history](img/history.jpg)

Es pseudocódigo, el modelo de archivos e historia de `Git`se puede explicar de la siguiente manera:

**Tipos de estructuras:**
```python
type blob = array<byte>    # file = coleccion de bytes

type tree = map<string, tree | blob> # folder = mapea un nombre de directorio a contenido (archivos o folders)

type commit = struct{           # metadata con la información de la fotografía
    parents: array<commit>
    author: string
    message: string
    snapshot: tree  # esta es una referencia al objeto no el objeto en si
```
**Almacenamiento en disco:**los objetos son los que Git almacena en disco
```python
type object = blog | tree | commit

objects = map<string, objects>

# para almacenar en disco se utiliza el hash del objeto
def store(o)
    id = sha1(o) # hash del objeto
    objects[id] = o

}

df load(id)
    return objects[id]
```
**Referencias**:
mapea referencias de un nombre legible para el humano con su hash respectivo
```python
references = map<string, string>
```

La historia y los hash son inmutables, pero las referencias sí se pueden cambiar.
En forma general, todos los comandos de `Git` son manipulaciones de los datos de los *objetos* o de las *referencia*

## Demostración

Esta sera una demostración de cómo utilizar comandos de git, pero no de como hacer la inicializacion para no alterar el repositorio ya existente.

Al visualizar el directorio `.git` podemos `refs` y `objects` que son donde `Git` almacena la información de los repositorios.

In [6]:
%sx ls ../../.git

['branches',
 'config',
 'description',
 'HEAD',
 'hooks',
 'index',
 'info',
 'logs',
 'objects',
 'packed-refs',
 'refs']

### git help
El comando `git help` se puede utilizar para obtener información sobre `git`. También se puede utilizar `git help <comando>` para tener información adicional del comando específico.

In [8]:
%sx git help

['usage: git [--version] [--help] [-C <path>] [-c <name>=<value>]',
 '           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]',
 '           [-p | --paginate | -P | --no-pager] [--no-replace-objects] [--bare]',
 '           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]',
 '           <command> [<args>]',
 '',
 'These are common Git commands used in various situations:',
 '',
 'start a working area (see also: git help tutorial)',
 '   clone             Clone a repository into a new directory',
 '   init              Create an empty Git repository or reinitialize an existing one',
 '',
 'work on the current change (see also: git help everyday)',
 '   add               Add file contents to the index',
 '   mv                Move or rename a file, a directory, or a symlink',
 '   restore           Restore working tree files',
 '   rm                Remove files from the working tree and from the index',
 '   sparse-checkout   Initialize and modify the spar

### git status
El comando `git status` se puede ver información de la historia y el estatus actual del repositorio.

In [1]:
%sx git status

['On branch main',
 "Your branch is up to date with 'origin/main'.",
 '',
 'nothing to commit, working tree clean']

El staging area es dónde le decimos a git qué cambios incluir en la próxima fotografía que tome.
Podemos ver cómo cambia el status cuándo se crea un nuevo archivo:

In [2]:
%sx echo 'archivo de ejemplo para git' > archivo_ejemplo.txt
%sx git status

['On branch main',
 "Your branch is up to date with 'origin/main'.",
 '',
 'Untracked files:',
 '  (use "git add <file>..." to include in what will be committed)',
 '\tarchivo_ejemplo.txt',
 '',
 'nothing added to commit but untracked files present (use "git add" to track)']

### git add
Usamos `git add` para empezar a llevar el tracking de un archivo.

Podemos ver cómo el status cambia al agregarlo al tracking de git

In [3]:
%sx git add archivo_ejemplo.txt
%sx git status

['On branch main',
 "Your branch is up to date with 'origin/main'.",
 '',
 'Changes to be committed:',
 '  (use "git restore --staged <file>..." to unstage)',
 '\tnew file:   archivo_ejemplo.txt',
 '']

### git commit
Para tomar la fotografía, se utiliza el comando `git commit`

In [4]:
%sx git commit

['[main 3698a48] nuevo commit para archivo_ejemplo',
 ' 1 file changed, 1 insertion(+)',
 ' create mode 100644 tareas/02-tarea/archivo_ejemplo.txt']

### git log
Se puede utilizar el comando `git log` para ver la historia de todos los cambios. Aquí utilizaré -1 para solo mostrar el último commit

In [5]:
%sx git log -1

['commit 3698a489300c0cd4b2baa9ae7a7237e5c9626753',
 'Author: Sofi GS <19873735+sofi-gt@users.noreply.github.com>',
 'Date:   Sat Feb 6 14:16:37 2021 -0600',
 '',
 '    nuevo commit para archivo_ejemplo']

### git cat-file
Para ver el contenido de un commit se puede utilizar el comando `git cat-file` con el hash

In [6]:
%sx git cat-file -p 83644be

['tree 56e4e6215bca85188f3d6b9d504abf0f01507667',
 'parent 946f83da08279b6d47ad9ef58562bf5b838dc313',
 'author Sofi GS <19873735+sofi-gt@users.noreply.github.com> 1612634949 -0600',
 'committer Sofi GS <19873735+sofi-gt@users.noreply.github.com> 1612634949 -0600',
 '',
 'Agregar archivo de ejemplo']

Los comandos `git add` y `git commit` están separados para poder tener mayor flexibilidad y organización. Yo puedo decidir qué archivos se incluyen en cada commit

### Otro ejemplo de staging

In [7]:
%sx echo "una nueva linea" >> archivo_ejemplo.txt

[]

In [8]:
%sx git status

['On branch main',
 "Your branch is ahead of 'origin/main' by 1 commit.",
 '  (use "git push" to publish your local commits)',
 '',
 'Changes not staged for commit:',
 '  (use "git add <file>..." to update what will be committed)',
 '  (use "git restore <file>..." to discard changes in working directory)',
 '\tmodified:   archivo_ejemplo.txt',
 '',
 'no changes added to commit (use "git add" and/or "git commit -a")']

In [9]:
%sx git add archivo_ejemplo.txt
%sx git status

['On branch main',
 "Your branch is ahead of 'origin/main' by 1 commit.",
 '  (use "git push" to publish your local commits)',
 '',
 'Changes to be committed:',
 '  (use "git restore --staged <file>..." to unstage)',
 '\tmodified:   archivo_ejemplo.txt',
 '']

In [10]:
%sx git commit

['[main 398dac2] nuevo ejemplo, agregar una linea',
 ' 1 file changed, 1 insertion(+)']

También podemos visualizar los logs de forma gráfica utilizando las opciones `--graph` y `--decorate`.

In [12]:
%sx git log -2 --graph --decorate

['* commit 398dac27b64aeb9c30e958cb1ff7ecd1cd72114d (HEAD -> main)',
 '| Author: Sofi GS <19873735+sofi-gt@users.noreply.github.com>',
 '| Date:   Sat Feb 6 14:17:34 2021 -0600',
 '| ',
 '|     nuevo ejemplo, agregar una linea',
 '| ',
 '* commit 3698a489300c0cd4b2baa9ae7a7237e5c9626753',
 '| Author: Sofi GS <19873735+sofi-gt@users.noreply.github.com>',
 '| Date:   Sat Feb 6 14:16:37 2021 -0600',
 '| ',
 '|     nuevo commit para archivo_ejemplo']

 `main` es una referencia que se refiere al rama principal de desarrollo. Por convención se refiere a la versión más actualizada del repositorio.

 `HEAD` es una referencia especial que se refiere a dónde estás viendo actualmente. Usualmente es el último commit

### git checkout
`git checkout` permite cambiar el estado del directorio al estado que tenía un commit previo.
Este mueve la referencia de `HEAD` al commit especificado y cambia los contenidos de los objetos a ese estado.

También se puede utilizar `git checkout <nombre_archivo>` para descartar las modificaciones del archivo y dejarlo como aparece en `HEAD`

In [13]:
%sx git checkout 3698a4893

["Note: switching to '3698a4893'.",
 '',
 "You are in 'detached HEAD' state. You can look around, make experimental",
 'changes and commit them, and you can discard any commits you make in this',
 'state without impacting any branches by switching back to a branch.',
 '',
 'If you want to create a new branch to retain commits you create, you may',
 'do so (now or later) by using -c with the switch command. Example:',
 '',
 '  git switch -c <new-branch-name>',
 '',
 'Or undo this operation with:',
 '',
 '  git switch -',
 '',
 'Turn off this advice by setting config variable advice.detachedHead to false',
 '',
 'HEAD is now at 3698a48 nuevo commit para archivo_ejemplo']

Aquí podemos observar que no se muestra la nueva línea del archivo de ejemplo

In [14]:
%sx cat archivo_ejemplo.txt

['archivo de ejemplo para git']

In [18]:
%sx git log --all --graph --decorate | head -n 10

['* commit 398dac27b64aeb9c30e958cb1ff7ecd1cd72114d (main)',
 '| Author: Sofi GS <19873735+sofi-gt@users.noreply.github.com>',
 '| Date:   Sat Feb 6 14:17:34 2021 -0600',
 '| ',
 '|     nuevo ejemplo, agregar una linea',
 '| ',
 '* commit 3698a489300c0cd4b2baa9ae7a7237e5c9626753 (HEAD)',
 '| Author: Sofi GS <19873735+sofi-gt@users.noreply.github.com>',
 '| Date:   Sat Feb 6 14:16:37 2021 -0600',
 '| ']

En este último log se puede ver que el `HEAD` no está en el último commit.
Para regresar al último commit se puede utilizar `git checkout` utilizando la referencia `main`

In [19]:
%sx git checkout main

['Previous HEAD position was 3698a48 nuevo commit para archivo_ejemplo',
 "Switched to branch 'main'",
 "Your branch is ahead of 'origin/main' by 2 commits.",
 '  (use "git push" to publish your local commits)']

Ahora ya aparece la línea que habíamos agregado

In [20]:
%sx cat archivo_ejemplo.txt

['archivo de ejemplo para git', 'una nueva linea']

### git diff
Muestra lo cambios que existen comparados con el commit anterior o con un commit especificado.

In [21]:
%sx git diff archivo_ejemplo.txt

[]

Se puede ver que cambió un un commit específico a otro utilizando sus referencias o hash

In [22]:
%sx git diff 3698a48930 HEAD archivo_ejemplo.txt

['diff --git a/tareas/02-tarea/archivo_ejemplo.txt b/tareas/02-tarea/archivo_ejemplo.txt',
 'index baa80e8..20d7b20 100644',
 '--- a/tareas/02-tarea/archivo_ejemplo.txt',
 '+++ b/tareas/02-tarea/archivo_ejemplo.txt',
 '@@ -1 +1,2 @@',
 ' archivo de ejemplo para git',
 '+una nueva linea']

## Branching y Merging

Para demostrar la funcionalidad de branching utilizaremos un script de python. Vamos a agregar este script al tracking de git y realizando un commit

In [29]:
%sx cat programa-ejemplo.py

['import sys',
 '',
 'def default():',
 "    print('hello')",
 '',
 'def main():',
 '    default()',
 '',
 "if __name__ == '__main__':",
 '    main()']

In [30]:
%sx git status

['On branch main',
 "Your branch is up to date with 'origin/main'.",
 '',
 'Untracked files:',
 '  (use "git add <file>..." to include in what will be committed)',
 '\tprograma-ejemplo.py',
 '',
 'nothing added to commit but untracked files present (use "git add" to track)']

In [31]:
%sx git add programa-ejemplo.py

[]

In [32]:
%sx git commit

['[main 73c55b6] programa ejemplo branching segundo intento',
 ' 1 file changed, 10 insertions(+)',
 ' create mode 100644 tareas/02-tarea/programa-ejemplo.py']

### git branch
Lista todos las branches presentes en el repositorio.

In [33]:
%sx git branch -vv

['* main 73c55b6 [origin/main: ahead 1] programa ejemplo branching segundo intento']

Al usar una referencia después de `branch`, git crea una referencia en el lugar en donde estamos (en el commit actual)

In [34]:
%sx git branch ejemplo
%sx git log -2 --graph --decorate

['* commit 73c55b66b7bc07118d6972738d5060516ba9d0e2 (HEAD -> main, ejemplo)',
 '| Author: Sofi GS <19873735+sofi-gt@users.noreply.github.com>',
 '| Date:   Sat Feb 6 14:42:04 2021 -0600',
 '| ',
 '|     programa ejemplo branching',
 '|     segundo intento',
 '| ',
 '* commit 18c2a8c55c3287295b57cbf39f100205826d39e2 (origin/main, origin/HEAD)',
 '| Author: Sofi GS <19873735+sofi-gt@users.noreply.github.com>',
 '| Date:   Sat Feb 6 14:40:56 2021 -0600',
 '| ',
 '|     correcciones branching']

Para cambiar de branch, se utiliza el comando `git checkout`

In [35]:
%sx git checkout ejemplo

["Switched to branch 'ejemplo'"]

Ahora `HEAD` apunta hacia el branch ejemplo

In [36]:
%sx git log -2 --graph --decorate

['* commit 73c55b66b7bc07118d6972738d5060516ba9d0e2 (HEAD -> ejemplo, main)',
 '| Author: Sofi GS <19873735+sofi-gt@users.noreply.github.com>',
 '| Date:   Sat Feb 6 14:42:04 2021 -0600',
 '| ',
 '|     programa ejemplo branching',
 '|     segundo intento',
 '| ',
 '* commit 18c2a8c55c3287295b57cbf39f100205826d39e2 (origin/main, origin/HEAD)',
 '| Author: Sofi GS <19873735+sofi-gt@users.noreply.github.com>',
 '| Date:   Sat Feb 6 14:40:56 2021 -0600',
 '| ',
 '|     correcciones branching']

Agregamos funcionalidad al script y con `git diff` podemos observar los cambios realizados.

In [37]:
%sx git diff

['diff --git a/tareas/02-tarea/programa-ejemplo.py b/tareas/02-tarea/programa-ejemplo.py',
 'index 433a0a1..fd4a0f3 100644',
 '--- a/tareas/02-tarea/programa-ejemplo.py',
 '+++ b/tareas/02-tarea/programa-ejemplo.py',
 '@@ -1,10 +1,16 @@',
 ' import sys',
 ' ',
 '+def cat():',
 "+    print('meaow')",
 '+',
 ' def default():',
 "     print('hello')",
 ' ',
 ' def main():',
 '-    default()',
 "+    if sys.argv[1] == 'cat':",
 '+        cat()',
 '+    else:',
 '+        default()',
 ' ',
 " if __name__ == '__main__':",
 '     main()',
 '\\ No newline at end of file']

In [38]:
%sx git add programa-ejemplo.py
%sx git commit

['[ejemplo a55d3c0] agregar funcionalidad a script se agrega funcionalidad para cat',
 ' 1 file changed, 7 insertions(+), 1 deletion(-)']

In [39]:
%sx git log -3 --graph --decorate --oneline

['* a55d3c0 (HEAD -> ejemplo) agregar funcionalidad a script se agrega funcionalidad para cat',
 '* 73c55b6 (main) programa ejemplo branching segundo intento',
 '* 18c2a8c (origin/main, origin/HEAD) correcciones branching']

Si nos regresamos al `main` branch veremos que los cambios no están en este branch.

In [40]:
%sx git checkout main
%sx cat programa-ejemplo.py

['import sys',
 '',
 'def default():',
 "    print('hello')",
 '',
 'def main():',
 '    default()',
 '',
 "if __name__ == '__main__':",
 '    main()']

Ahora veremos un ejemplo de trabajar en dos branches en paralelo creando y posicionandonos en la nueva branch

In [44]:
%sx git checkout -b ejemplo2

["Switched to a new branch 'ejemplo2'"]

In [45]:
%sx git log -3 --graph --decorate --oneline

['* 73c55b6 (HEAD -> ejemplo2, main) programa ejemplo branching segundo intento',
 '* 18c2a8c (origin/main, origin/HEAD) correcciones branching',
 '* 08d4fab script de python para ejemplo de merging y branching']

In [46]:
%sx git diff

['diff --git a/tareas/02-tarea/programa-ejemplo.py b/tareas/02-tarea/programa-ejemplo.py',
 'index 433a0a1..625bc52 100644',
 '--- a/tareas/02-tarea/programa-ejemplo.py',
 '+++ b/tareas/02-tarea/programa-ejemplo.py',
 '@@ -3,8 +3,14 @@ import sys',
 ' def default():',
 "     print('hello')",
 ' ',
 '+def dog():',
 "+    print('woof')",
 '+',
 ' def main():',
 '-    default()',
 "+    if sys.argv[1] == 'dog':",
 '+        dog()',
 '+    else:',
 '+        default()',
 ' ',
 " if __name__ == '__main__':",
 '     main()',
 '\\ No newline at end of file']

In [47]:
%sx git add programa-ejemplo.py
%sx git commit

['[ejemplo2 b558519] se agrego funcionalidad de perro',
 ' 1 file changed, 7 insertions(+), 1 deletion(-)']

In [52]:
%sx git log --all --graph --decorate --oneline | head -n 5

['* b558519 (HEAD -> ejemplo2) se agrego funcionalidad de perro',
 '| * a55d3c0 (ejemplo) agregar funcionalidad a script se agrega funcionalidad para cat',
 '|/  ',
 '* 73c55b6 (main) programa ejemplo branching segundo intento',
 '* 18c2a8c (origin/main, origin/HEAD) correcciones branching']

### git merge
Permite combinar branches.

Nos vamos a posicionar en el main branch y desde allí combinaremos las dos branches nuevas que hicimos.

In [53]:
%sx git checkout main

["Switched to branch 'main'",
 "Your branch is ahead of 'origin/main' by 1 commit.",
 '  (use "git push" to publish your local commits)']

In [55]:
%sx git merge ejemplo

['Updating 73c55b6..a55d3c0',
 'Fast-forward',
 ' tareas/02-tarea/programa-ejemplo.py | 8 +++++++-',
 ' 1 file changed, 7 insertions(+), 1 deletion(-)']

In [58]:
%sx git log --all --graph --decorate --oneline | head -n 5

['* b558519 (ejemplo2) se agrego funcionalidad de perro',
 '| * a55d3c0 (HEAD -> main, ejemplo) agregar funcionalidad a script se agrega funcionalidad para cat',
 '|/  ',
 '* 73c55b6 programa ejemplo branching segundo intento',
 '* 18c2a8c (origin/main, origin/HEAD) correcciones branching']

Se puede ver que el branch main y el branch ejemplo apuntan al mismo commit.
Y si vemos el script de ejemplo, tiene incorporada la funcionalidad de cat en el main branch

In [64]:
%sx cat programa-ejemplo.py

['import sys',
 '',
 'def cat():',
 "    print('meaow')",
 '',
 'def default():',
 "    print('hello')",
 '',
 'def main():',
 "    if sys.argv[1] == 'cat':",
 '        cat()',
 '    else:',
 '        default()',
 '',
 "if __name__ == '__main__':",
 '    main()']

Al intentar combinar el branch con la funcionalidad de dog, se genera un conflicto.
No se puede hacer un "fast foward" como en el merge anterior

In [65]:
%sx git merge ejemplo2

['Auto-merging tareas/02-tarea/programa-ejemplo.py',
 'CONFLICT (content): Merge conflict in tareas/02-tarea/programa-ejemplo.py',
 'Automatic merge failed; fix conflicts and then commit the result.']

El conflicto se puede resolver de forma manual o se puede utilizar una herramienta como `git mergetools`
Para este ejemplo, se corrigió manualmente y se continuo el merge.

In [68]:
%sx cat programa-ejemplo.py

['import sys',
 '',
 'def cat():',
 "    print('meaow')",
 '',
 'def default():',
 "    print('hello')",
 '',
 'def dog():',
 "    print('woof')",
 '',
 'def main():',
 "    if sys.argv[1] == 'cat':",
 '        cat()',
 "    elif sys.argv[1] == 'dog':",
 '        dog()',
 '    else:',
 '        default()',
 '',
 "if __name__ == '__main__':",
 '    main()']

In [72]:
%sx git log --all --graph --decorate --oneline | head -n 8

["*   e57e165 (HEAD -> main) Merge branch 'ejemplo2' into main",
 '|\\  ',
 '| * b558519 (ejemplo2) se agrego funcionalidad de perro',
 '* | a55d3c0 (ejemplo) agregar funcionalidad a script se agrega funcionalidad para cat',
 '|/  ',
 '* 73c55b6 programa ejemplo branching segundo intento',
 '* 18c2a8c (origin/main, origin/HEAD) correcciones branching',
 '* 08d4fab script de python para ejemplo de merging y branching']

## Remotes

Esta parte no será una demostración sino solo notas de los comandos utilizados para configurar repositorios remotos como Github.

Estos repositorios son identifcados por un nombre y un url.

### git remote add
Agrega un repositorio remoto al repositorio actual.

```bash
git remote add <nombre> <url>
```

### git push
Enviar cambios del repositorio local al repositorio remoto. Si no existe un branch remoto, este comando la creará.

```bash
git push <remote> <local branch>:<remote branch>
```

### git clone
Se utiliza para copiar un repositorio remoto y descargarlo a un folder local

```bash
git clone <url> <nombre folder destino>
```

### relacionar branch local con remote
Para facilitar git push, se puede utilizar el siguiente comando para relacionar el branch actual con un branch remoto.

```bash
git branch --set-upstream-to=origin/main
```
con esto se puede utilizar el shortcut
```bash
git push
```

### git fetch
Fetch no cambia nada de la historia local, solo va a traer información de los cambios que hay en el remoto.

```bash
git fetch <nombre remote>
```

### git pull
Es la combinación de `git fetch` y `git pull`.

```bash
git pull
```